In [1]:
import polars as pl
from influxdb_client.client.influxdb_client import InfluxDBClient
from influxdb_client import __version__ as influxdb_version
from datetime import datetime, timedelta

# InfluxDB connection
url = "http://192.168.1.17:8086"
token = "_gCwoPkdKM6aF-SXB9crHatd4lKFBXKVcBHhyHMgCqTI2GvsxWkO6qooThciZp49Ra-dQR9B6jOd_ve5isMcHw=="
org = "mush"
bucket = "bucket"

client = InfluxDBClient(url=url, token=token, org=org)
query_api = client.query_api()

In [2]:
# Find all temperature measurements
structure_query = '''
from(bucket: "bucket")
  |> range(start: -2d)
  |> filter(fn: (r) => r["topic"] =~ /temperature$/)
  |> distinct(column: "topic")
'''

result = query_api.query(structure_query)

print("Temperature sensors found:")
for table in result:
    for record in table.records:
        print(f"  - {record.get_value()}")

Temperature sensors found:
  - mush/controllers/C1/sensors/bme280_0/temperature
  - mush/controllers/C1/sensors/bme280_1/temperature
  - mush/controllers/C1/sensors/dht_0/temperature
  - mush/controllers/C1/sensors/ds18b20_0/temperature
  - mush/controllers/C1/sensors/ds18b20_1/temperature
  - mush/controllers/C1/sensors/sht_0/temperature
  - mush/controllers/C3/sensors/scd_0/temperature


In [ ]:
# Get temperature data for all sensors
query = '''
from(bucket: "bucket")
  |> range(start: -2d)
  |> filter(fn: (r) => r["topic"] =~ /temperature$/)
  |> pivot(rowKey:["_time"], columnKey: ["topic"], valueColumn: "_value")
'''

result = query_api.query(query)

# Convert to Polars DataFrame
records = []
for table in result:
    for record in table.records:
        row = {
            'time': record.get_time(),
            **{k: v for k, v in record.values.items() if not k.startswith('_')}
        }
        records.append(row)

df = pl.DataFrame(records)

print("Data shape:", df.shape)
print("\nColumns:", df.columns)
print("\nFirst few rows:")
df.head()